In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [124]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [125]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [12]:
# Проверяем пропуски
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отдели целевой признак, раздели обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируй данные с помощью StandardScaler.**

In [127]:
# y - целевой признак
y = data['quality']


X_train, X_holdout, y_train, y_holdout = train_test_split(data, y, test_size = 0.30, random_state = 17)

# Проверяем шейпы, чтобы убедиться, что разбивка правильная
#(4898, 12)
data.shape
#(3428, 12)
X_train.shape
#(1470, 12)
X_holdout.shape

X_train.drop(['quality'], axis=1,inplace=True)
X_holdout.drop(['quality'], axis=1,inplace=True)

In [128]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучи простую линейную регрессию.**

In [129]:
linreg = LinearRegression(n_jobs=4)
linreg.fit(X_train_scaled, y_train)

# Предсказание на обучающей выборке
predicted_on_train = linreg.predict(X_train_scaled)

# Предсказание на отложенной выборке
predicted_on_holdout = linreg.predict(X_holdout_scaled)

# Проверка размерности
# predicted_on_train.shape
# predicted_on_holdout.shape

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [130]:
# Метрика качества RMSE
print("Mean squared error on train:{0}".format(np.sqrt(mean_squared_error(predicted_on_train, y_train))))
print("Mean squared error on test:{0}".format(np.sqrt(mean_squared_error(predicted_on_holdout, y_holdout))))

Mean squared error on train:0.7470345701373919
Mean squared error on test:0.7643607199748388


**Посмотри на коэффициенты модели и отранжируй признаки по влиянию на качество вина (большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создай для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [120]:
linreg.coef_


array([ 9.78219223e-02, -1.92259947e-01, -1.83224449e-04,  5.38164096e-01,
        8.12724353e-03,  4.21804406e-02,  1.43040227e-02, -6.65720472e-01,
        1.50036006e-01,  6.20533605e-02,  1.29533447e-01])

In [141]:
data.columns
df_1 = pd.DataFrame(abs(linreg.coef_), X_train.columns).sort_values(max)

## Lasso-регрессия

**Обучи Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [142]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотри на коэффициенты модели и отранжируй признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [143]:
df_2 = pd.DataFrame(abs(lasso1.coef_), X_train.columns).sort_values(max)

**Теперь определи лучшее значение $\alpha$ в процессе кросс-валидации на 5 фолдах. Используй LassoCV и random_state=17.**

In [157]:
alphas = np.logspace(-6, 2, 20)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

# Предсказание на обучающей выборке
lasso_cv_predicted_on_train = lasso_cv.predict(X_train_scaled)

# Предсказание на отложенной выборке
lasso_cv_predicted_on_holdout = lasso_cv.predict(X_holdout_scaled)



In [158]:
lasso_cv.alpha_

0.0003359818286283781

**Выведи коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [159]:
df_3 = pd.DataFrame(abs(lasso_cv.coef_), X_train.columns).sort_values(max)

**Оцени среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [162]:
print("Mean squared error on train:{0}".format(np.sqrt(mean_squared_error(lasso_cv_predicted_on_train, y_train))))
print("Mean squared error on test:{0}".format(np.sqrt(mean_squared_error(lasso_cv_predicted_on_holdout, y_holdout))))

Mean squared error on train:0.7470433731260416
Mean squared error on test:0.7636269979668784


In [155]:
result = pd.DataFrame([list(i.index) for i in [df_1, df_2, df_3]]).T

In [156]:
result

,0,1,2
0,citric acid,fixed acidity,citric acid
1,chlorides,citric acid,chlorides
2,total sulfur dioxide,total sulfur dioxide,total sulfur dioxide
3,free sulfur dioxide,chlorides,free sulfur dioxide
4,sulphates,sulphates,sulphates
5,fixed acidity,free sulfur dioxide,fixed acidity
6,alcohol,pH,alcohol
7,pH,volatile acidity,pH
8,volatile acidity,density,volatile acidity
9,residual sugar,residual sugar,residual sugar
